# W&B Image Normalization Demo

This notebook demonstrates how `wandb.Image` automatically normalizes different types of image data and how to control this behavior.

## What you'll learn:
- How `wandb.Image` normalizes PyTorch tensors and NumPy arrays
- When normalization is applied vs when it's not
- How to avoid unwanted normalization
- Best practices for image logging

## Setup

First, let's install the required dependencies and import the necessary libraries.

In [ ]:
# Install required packages
!pip install wandb torch torchvision pillow matplotlib numpy

In [ ]:
import wandb
import torch
import numpy as np
from PIL import Image as PILImage
import matplotlib.pyplot as plt

# Set up matplotlib for better visualization
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## Initialize W&B

Let's start a W&B run to log our examples.

In [ ]:
# Initialize W&B run
run = wandb.init(
    project="image-normalization-demo",
    name="normalization-examples",
    config={
        "description": "Demonstrating wandb.Image normalization behavior"
    }
)

## Understanding Image Normalization

When you pass PyTorch tensors or NumPy arrays to `wandb.Image`, the pixel values are automatically normalized to the range [0, 255] unless you set `normalize=False`.

**Normalization is applied to:**
- PyTorch tensors (format: `(channel, height, width)`)
- NumPy arrays (format: `(height, width, channel)`)

**Normalization is NOT applied to:**
- PIL Images (passed as-is)
- File paths (loaded as-is)

**Normalization algorithm:**
- [0, 1] range: values are multiplied by 255
- [-1, 1] range: values are rescaled using `255 * 0.5 * (data + 1)`
- Other ranges: values are clipped to [0, 255]

## Example 1: [0, 1] Range Data

When your tensor/array values are in the [0, 1] range, `wandb.Image` will multiply all values by 255.

In [ ]:
# Create test data in [0, 1] range
data_0_1 = np.random.rand(64, 64, 3)
print(f"Original data range: [{data_0_1.min():.3f}, {data_0_1.max():.3f}]")

# Convert to PyTorch tensor (channel, height, width format)
tensor_0_1 = torch.from_numpy(data_0_1).permute(2, 0, 1).float()
print(f"Tensor shape: {tensor_0_1.shape}")
print(f"Tensor range: [{tensor_0_1.min():.3f}, {tensor_0_1.max():.3f}]")

# Visualize the original data
plt.figure(figsize=(8, 6))
plt.imshow(data_0_1)
plt.title(f'[0, 1] Range Data\nValues will be multiplied by 255')
plt.colorbar()
plt.axis('off')
plt.show()

# Log to W&B
wandb.log({
    "example_0_1_range": wandb.Image(
        tensor_0_1,
        caption="[0, 1] range tensor - values will be multiplied by 255"
    )
})

## Example 2: [-1, 1] Range Data

When your tensor/array values are in the [-1, 1] range, `wandb.Image` will rescale them using the formula: `255 * 0.5 * (data + 1)`

In [ ]:
# Create test data in [-1, 1] range
data_neg1_1 = np.random.rand(64, 64, 3) * 2 - 1
print(f"Original data range: [{data_neg1_1.min():.3f}, {data_neg1_1.max():.3f}]")

# Convert to PyTorch tensor
tensor_neg1_1 = torch.from_numpy(data_neg1_1).permute(2, 0, 1).float()
print(f"Tensor shape: {tensor_neg1_1.shape}")
print(f"Tensor range: [{tensor_neg1_1.min():.3f}, {tensor_neg1_1.max():.3f}]")

# Visualize the original data
plt.figure(figsize=(8, 6))
plt.imshow(data_neg1_1, cmap='RdBu_r')
plt.title(f'[-1, 1] Range Data\nValues will be rescaled: -1→0, 0→127.5, 1→255')
plt.colorbar()
plt.axis('off')
plt.show()

# Log to W&B
wandb.log({
    "example_neg1_1_range": wandb.Image(
        tensor_neg1_1,
        caption="[-1, 1] range tensor - values will be rescaled"
    )
})

## Example 3: Avoiding Normalization with PIL Images

To avoid normalization, you can convert your tensors to PIL Images before passing them to `wandb.Image`.

In [ ]:
# Create tensor with values in [0, 1] range
tensor_0_1 = torch.rand(3, 64, 64)
print(f"Tensor range: [{tensor_0_1.min():.3f}, {tensor_0_1.max():.3f}]")

# Convert to PIL Image to avoid normalization
pil_image = PILImage.fromarray(
    (tensor_0_1.permute(1, 2, 0).numpy() * 255).astype('uint8')
)
print(f"PIL Image size: {pil_image.size}")
print(f"PIL Image mode: {pil_image.mode}")

# Visualize the PIL image
plt.figure(figsize=(8, 6))
plt.imshow(pil_image)
plt.title('PIL Image - No normalization applied')
plt.axis('off')
plt.show()

# Log to W&B
wandb.log({
    "example_pil_no_normalization": wandb.Image(
        pil_image,
        caption="PIL Image - no normalization applied"
    )
})

## Example 4: Using normalize=False

You can also disable normalization by setting `normalize=False`. Values will be clipped to [0, 255].

In [ ]:
# Create tensor with values in [0, 1] range
tensor_0_1 = torch.rand(3, 64, 64)
print(f"Tensor range: [{tensor_0_1.min():.3f}, {tensor_0_1.max():.3f}]")

# Disable normalization
wandb.log({
    "example_normalize_false": wandb.Image(
        tensor_0_1,
        normalize=False,
        caption="Normalization disabled - values will be clipped to [0, 255]"
    )
})

# Also log with normal normalization for comparison
wandb.log({
    "example_normalize_true": wandb.Image(
        tensor_0_1,
        normalize=True,
        caption="Normalization enabled - values will be multiplied by 255"
    )
})

print("Logged both normalized and non-normalized versions for comparison")

## Best Practices

Based on what we've learned, here are some best practices for working with `wandb.Image`:

### 1. **For consistent results**: Pre-process your data to the expected [0, 255] range before logging
### 2. **To avoid normalization**: Convert tensors to PIL Images using `PILImage.fromarray()`
### 3. **For debugging**: Use `normalize=False` to see the raw values (they will be clipped to [0, 255])
### 4. **For precise control**: Use PIL Images when you need exact pixel values

### Common Issues to Watch Out For:
- **Unexpected brightness**: If your tensor values are in [0, 1] range, they will be multiplied by 255, making the image much brighter
- **Data loss**: Values outside the [0, 255] range will be clipped, potentially losing information
- **Inconsistent behavior**: Different input types (tensor vs PIL vs file path) may produce different results

In [ ]:
# Finish the W&B run
wandb.finish()
print("✅ Demo completed! Check your W&B dashboard to see all the logged images.")